In [1]:
using LinearAlgebraicRepresentation
Lar = LinearAlgebraicRepresentation
using BenchmarkTools

## Funzione da ottimizzare

In [2]:
function linefragments(V,EV,Sigma)
    m = length(Sigma) 
    sigma = map(sort,Sigma) 
    reducedsigma = sigma 
    params = Array{Float64,1}[[] for i=1:m]
    for h=1:m
        if sigma[h] ≠ []
            line1 = V[:,EV[h]]
            for k in sigma[h]
                line2 = V[:,EV[k]]
                out = intersection(line1,line2) 
                if out ≠ ()
                    α,β = out
                    if 0<=α<=1 && 0<=β<=1
                        push!(params[h], α)
                        push!(params[k], β)
                    end
                end
            end
        end
    end
    fragparams = []
    for line in params
        push!(line, 0.0, 1.0)
        line = sort(collect(Set(line)))
        push!(fragparams, line)
    end
    return fragparams
end

linefragments (generic function with 1 method)

## Dipendenze della funzione

In [3]:
function intersection(line1,line2)
    x1,y1,x2,y2 = vcat(line1...)
    x3,y3,x4,y4 = vcat(line2...)

    det = (x4-x3)*(y1-y2)-(x1-x2)*(y4-y3)
    if det != 0.0
        a = 1/det
        b = [y1-y2 x2-x1; y3-y4 x4-x3]  # x1-x2 => x2-x1 bug in the source link !!
        c = [x1-x3; y1-y3]
        (β,α) = a * b * c
    else
        return ()
    end
    return α,β
end

intersection (generic function with 1 method)

## Dati in input

In [4]:
V = [1.0 2.0 3.0 4.0 5.0 6.0 7.0 8.0 9.0 10.0 11.0 12.0 13.0 14.0 15.0 16.0 17.0 18.0 19.0 20.0 21.0 22.0 23.0 24.0 25.0 26.0 27.0 28.0 29.0 30.0 31.0 32.0 33.0 34.0 35.0 36.0 37.0 38.0 39.0 40.0 41.0 42.0 43.0 44.0 45.0 46.0 47.0 48.0 49.0 50.0 51.0 52.0 53.0 54.0 55.0 56.0 57.0 58.0 59.0 60.0 61.0 62.0 63.0 64.0 65.0 66.0 67.0 68.0 69.0 70.0 71.0 72.0 73.0 74.0 75.0 76.0 77.0 78.0 79.0 80.0 81.0 82.0 83.0 84.0 85.0 86.0 87.0 88.0 89.0 90.0 91.0 92.0 93.0 94.0 95.0 96.0 97.0 98.0 99.0 100.0 101.0 102.0 103.0 104.0 105.0 106.0 107.0 108.0 109.0 110.0 111.0 112.0 113.0 114.0 115.0 116.0 117.0 118.0 119.0 120.0 121.0 122.0 123.0 124.0 125.0 126.0 127.0 128.0 129.0;
     1.0 2.0 3.0 4.0 5.0 6.0 7.0 8.0 9.0 10.0 11.0 12.0 13.0 14.0 15.0 16.0 17.0 18.0 19.0 20.0 21.0 22.0 23.0 24.0 25.0 26.0 27.0 28.0 29.0 30.0 31.0 32.0 33.0 34.0 35.0 36.0 37.0 38.0 39.0 40.0 41.0 42.0 43.0 44.0 45.0 46.0 47.0 48.0 49.0 50.0 51.0 52.0 53.0 54.0 55.0 56.0 57.0 58.0 59.0 60.0 61.0 62.0 63.0 64.0 65.0 66.0 67.0 68.0 69.0 70.0 71.0 72.0 73.0 74.0 75.0 76.0 77.0 78.0 79.0 80.0 81.0 82.0 83.0 84.0 85.0 86.0 87.0 88.0 89.0 90.0 91.0 92.0 93.0 94.0 95.0 96.0 97.0 98.0 99.0 100.0 101.0 102.0 103.0 104.0 105.0 106.0 107.0 108.0 109.0 110.0 111.0 112.0 113.0 114.0 115.0 116.0 117.0 118.0 119.0 120.0 121.0 122.0 123.0 124.0 125.0 126.0 127.0 128.0 129.0]

EV = [[1,2],[3,4],[5,6],[7,8],[9,10],[11,12],[13,14],[15,16],[17,18],[19,20],[21,22],[23,24],[25,26],[27,28],[29,30],[31,32],[33,34],[35,36],[37,38],[39,40],[41,42],[43,44],[45,46],[47,48],[49,50],[51,52],[53,54],[55,56],[57,58],[59,60],[61,62],[63,64],[65,66],[67,68],[69,70],[71,72],[73,74],[75,76],[77,78],[79,80],[81,82],[83,84],[85,86],[87,88],[89,90],[91,92],[93,94],[95,96],[97,98],[99,100],[101,102],[103,104],[105,106],[107,108],[109,110],[111,112],[113,114],[115,116],[117,118],[119,120]]

using OrderedCollections
using IntervalTrees
function spaceindex(model::Lar.LAR)::Array{Array{Int,1},1}
    V,CV = model[1:2]
    # se il modello è in 3d o 2d (guardo le righe di V, in 3d V è una 3xN, in 2d V è una 2xN)
    dim = size(V,1)
    cellpoints = [ V[:,CV[k]]::Lar.Points for k=1:length(CV) ]
    #----------------------------------------------------------
    bboxes = [hcat(boundingbox(cell)...) for cell in cellpoints]
    xboxdict = coordintervals(1,bboxes)
    yboxdict = coordintervals(2,bboxes)
    # xs,ys are IntervalTree type
    xs = IntervalTrees.IntervalMap{Float64, Array}()
    for (key,boxset) in xboxdict
        xs[tuple(key...)] = boxset
    end
    ys = IntervalTrees.IntervalMap{Float64, Array}()
    for (key,boxset) in yboxdict
        ys[tuple(key...)] = boxset
    end
    xcovers = boxcovering(bboxes, 1, xs)
    ycovers = boxcovering(bboxes, 2, ys)
    covers = [intersect(pair...) for pair in zip(xcovers,ycovers)]

    if dim == 3
        zboxdict = coordintervals(3,bboxes)
        zs = IntervalTrees.IntervalMap{Float64, Array}()
        for (key,boxset) in zboxdict
            zs[tuple(key...)] = boxset
        end
        zcovers = boxcovering(bboxes, 3, zs)
        covers = [intersect(pair...) for pair in zip(zcovers,covers)]
    end
    # remove each cell from its cover
    for k=1:length(covers)
        covers[k] = setdiff(covers[k],[k])
    end
    return covers
end

function boundingbox(vertices::Lar.Points)
   minimum = mapslices(x->min(x...), vertices, dims=2)
   maximum = mapslices(x->max(x...), vertices, dims=2)
   return minimum, maximum
end

function coordintervals(coord,bboxes)
    boxdict = OrderedDict{Array{Float64,1},Array{Int64,1}}()
    for (h,box) in enumerate(bboxes)
        key = box[coord,:]
        if haskey(boxdict,key) == false
            boxdict[key] = [h]
        else
            push!(boxdict[key], h)
        end
    end
    return boxdict
end

function boxcovering(bboxes, index, tree)
    covers = [[] for k=1:length(bboxes)]
    for (i,boundingbox) in enumerate(bboxes)
        extent = bboxes[i][index,:]
        iterator = IntervalTrees.intersect(tree, tuple(extent...))
        for x in iterator
            append!(covers[i],x.value)
        end
    end
    return covers
end

spaceIndex = spaceindex((V,EV))

60-element Array{Array{Int64,1},1}:
 []
 []
 []
 []
 []
 []
 []
 []
 []
 []
 []
 []
 []
 ⋮
 []
 []
 []
 []
 []
 []
 []
 []
 []
 []
 []
 []

## 0) Benchmark vecchia funzione 

In [5]:
@btime linefragments(V,EV,spaceIndex)

  25.197 μs (668 allocations: 63.20 KiB)


60-element Array{Any,1}:
 [0.0, 1.0]
 [0.0, 1.0]
 [0.0, 1.0]
 [0.0, 1.0]
 [0.0, 1.0]
 [0.0, 1.0]
 [0.0, 1.0]
 [0.0, 1.0]
 [0.0, 1.0]
 [0.0, 1.0]
 [0.0, 1.0]
 [0.0, 1.0]
 [0.0, 1.0]
 ⋮
 [0.0, 1.0]
 [0.0, 1.0]
 [0.0, 1.0]
 [0.0, 1.0]
 [0.0, 1.0]
 [0.0, 1.0]
 [0.0, 1.0]
 [0.0, 1.0]
 [0.0, 1.0]
 [0.0, 1.0]
 [0.0, 1.0]
 [0.0, 1.0]

## 1) Controllo se la funzione è type unstable

In [6]:
@code_warntype linefragments(V,EV,spaceIndex)

Variables
  #self#::Core.Compiler.Const(linefragments, false)
  V::Array{Float64,2}
  EV::Array{Array{Int64,1},1}
  Sigma::Array{Array{Int64,1},1}
  m::Int64
  sigma::Array{Array{Int64,1},1}
  reducedsigma::Array{Array{Int64,1},1}
  params::Array{Array{Float64,1},1}
  @_9::Union{Nothing, Tuple{Int64,Int64}}
  fragparams::Array{Any,1}
  @_11::Union{Nothing, Tuple{Array{Float64,1},Int64}}
  @_12::Array{Array{Float64,1},1}
  @_13::Int64
  @_14::Union{Nothing, Tuple{Int64,Int64}}
  i::Int64
  h::Int64
  line1::Array{Float64,2}
  @_18::Union{Nothing, Tuple{Int64,Int64}}
  k::Int64
  @_20::Int64
  line2::Array{Float64,2}
  out::Tuple
  α::Any
  β::Any
  line::Array{Float64,1}
  @_26::Any
  @_27::Any
  @_28::Any

Body::Array{Any,1}
1 ──        Core.NewvarNode(:(params))
│           Core.NewvarNode(:(@_9))
│           Core.NewvarNode(:(fragparams))
│           Core.NewvarNode(:(@_11))
│           (m = Main.length(Sigma))
│           (sigma = Main.map(Main.sort, Sigma))
│           (reducedsigm

la funzione non  type unstable in quanto ho nell'output la stringa:

    Body::Array{Any,1}

## 2) Ottimizzazione cicli con @simd e @inbounds

In [7]:
using Base.Threads
function linefragments2(V,EV,Sigma)
    m = length(Sigma) 
    sigma = map(sort,Sigma) 
    reducedsigma = sigma 
    params = Array{Float64,1}[[] for i=1:m]
    @inbounds for h=1:m
        if sigma[h] ≠ []
            line1 = V[:,EV[h]]
            len = length(sigma[h])
            @inbounds @threads for k=1:len
                line2 = V[:,EV[sigma[h][k]]]
                out = intersection(line1,line2) 
                if out ≠ ()
                    α,β = out
                    if 0<=α<=1 && 0<=β<=1
                        push!(params[h], α)
                        push!(params[sigma[h][k]], β)
                    end
                end
            end
        end
    end
    fragparams = []
    lenp = length(params)
    @inbounds @threads for i=1:lenp
        push!(params[i], 0.0, 1.0)
        params[i] = sort(collect(Set(params[i])))
        push!(fragparams, params[i])
    end
    return fragparams
end
@btime linefragments2(V,EV,spaceIndex)

  24.385 μs (678 allocations: 64.72 KiB)


60-element Array{Any,1}:
 [0.0, 1.0]
 [0.0, 1.0]
 [0.0, 1.0]
 [0.0, 1.0]
 [0.0, 1.0]
 [0.0, 1.0]
 [0.0, 1.0]
 [0.0, 1.0]
 [0.0, 1.0]
 [0.0, 1.0]
 [0.0, 1.0]
 [0.0, 1.0]
 [0.0, 1.0]
 ⋮
 [0.0, 1.0]
 [0.0, 1.0]
 [0.0, 1.0]
 [0.0, 1.0]
 [0.0, 1.0]
 [0.0, 1.0]
 [0.0, 1.0]
 [0.0, 1.0]
 [0.0, 1.0]
 [0.0, 1.0]
 [0.0, 1.0]
 [0.0, 1.0]